In [15]:
using XLSX
using DataFrames
using OptimalTransportDataIntegration
using Distributions
using DocStringExtensions
using Parameters
using Printf
using DataFrames
using DataFrames, CategoricalArrays
using LinearAlgebra
using Flux
using Distances
using PythonOT
import Statistics: median
using Pkg
using FreqTables
using DataFrames, Statistics, StatsBase, MultivariateStats
using JuMP, Clp
tbl = XLSX.readtable("data_sansCSP.xlsx", 1)   # 1 = première feuille, ou utiliser le nom "Sheet1"
data = DataFrame(tbl)
eltype.(eachcol(data))
for c in names(data)
    data[!, c] = [ 
        x isa Number ? float(x) :                 # si déjà numérique → convertir en Float64
        x isa Missing ? missing :                 # missing → garder
        tryparse(Float64, String(x))              # si string → tenter de parser
        for x in data[!, c]
    ]
end


data = DataFrame(data)

Row,X1,X2,X3,X4,Y,Z,database
,Float64,Float64,Float64,Float64,Float64?,Float64?,Float64
1,36.3258,0.0,0.0,24.16,4.0,missing,1.0
2,30.3272,0.0,0.0,23.88,4.0,missing,1.0
3,40.2546,0.0,0.0,24.8,5.0,missing,1.0
4,38.3956,0.0,0.0,21.99,3.0,missing,1.0
5,35.2909,0.0,0.0,18.65,4.0,missing,1.0
6,41.1608,1.0,0.0,18.69,4.0,missing,1.0
7,37.3744,0.0,0.0,32.11,3.0,missing,1.0
8,22.5243,0.0,0.0,18.69,3.0,missing,1.0
9,38.0041,0.0,0.0,26.62,3.0,missing,1.0


In [16]:
describe(data, :nmissing)



Row,variable,nmissing
,Symbol,Int64
1,X1,0
2,X2,0
3,X3,0
4,X4,0
5,Y,3354
6,Z,991
7,database,0


In [17]:
for c in [:Y, :Z]
    data[!, c] = coalesce.(data[!, c], -1)
end

for c in [:Y, :Z, :database]
    data[!, c] = convert.(Union{Int64}, data[!, c])
end

In [59]:
println(data)

In [18]:
dba = subset(data, :database => ByRow(==(1)))
dbb = subset(data, :database => ByRow(==(2)))


cols = names(dba, r"^X")   

XA = transpose(Matrix(dba[:, cols]))
XB = transpose(Matrix(dbb[:, cols]))

Ylevels = 1:5
Zlevels = 1:6

YA = Flux.onehotbatch(dba.Y, Ylevels)
ZB = Flux.onehotbatch(dbb.Z, Zlevels)


6×3354 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 1  1  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅     ⋅  1  1  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  1  ⋅     1  ⋅  ⋅  1  1  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  1  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  1
 ⋅  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅

In [19]:


digitize(x, bins) = searchsortedlast.(Ref(bins), x)

XA = dba[:, cols]
XB = dbb[:, cols]

X = Vector{Int}[]
Xmdn = Vector{Float64}[]
cols = ["X1", "X4"]
for col in cols

        b = quantile(data[!, col], collect(0.25:0.25:0.75))
        bins = vcat(-Inf, b, +Inf)

        X1 = digitize(XA[!, col], bins)
        X2 = digitize(XB[!, col], bins)
        push!(X, vcat(X1, X2))

        X1mdn = zeros(Float32, size(X1, 1))
        for i in unique(X1)
            mdn = median(XA[X1 .== i, col])
            X1mdn[X1 .== i] .= mdn
        end

        X2mdn = zeros(Float32, size(X2, 1))
        for i in unique(X2)
            mdn = median(XB[X2 .== i, col])
            X2mdn[X2 .== i] .= mdn
        end

        push!(Xmdn, vcat(X1mdn, X2mdn))

end

cols = ["X2", "X3"]
for col in cols

        X1 = XA[!, col]
        X2 = XB[!, col]
        push!(X, vcat(X1, X2))

        X1mdn = XA[!, col]

        X2mdn = XB[!, col]

        push!(Xmdn, vcat(X1mdn, X2mdn))

end
X = stack(X)
Xmdn = stack(Xmdn)
X_hot = one_hot_encoder(X)


4345×10 Matrix{Int64}:
 0  0  0  1  0  0  1  0  0  0
 0  1  0  0  0  0  1  0  0  0
 0  0  0  1  0  0  0  1  0  0
 0  0  0  1  0  0  1  0  0  0
 0  0  0  1  1  0  0  0  0  0
 0  0  0  1  1  0  0  0  1  0
 0  0  0  1  0  0  0  1  0  0
 1  0  0  0  1  0  0  0  0  0
 0  0  0  1  0  0  0  1  0  0
 1  0  0  0  0  0  1  0  0  0
 ⋮              ⋮           
 0  0  0  1  0  0  0  1  1  0
 1  0  0  0  0  0  1  0  1  0
 0  0  0  1  0  0  0  1  0  0
 0  0  1  0  0  1  0  0  1  0
 0  1  0  0  1  0  0  0  0  0
 0  0  0  1  0  0  1  0  0  0
 0  0  1  0  0  1  0  0  1  0
 0  1  0  0  0  0  0  1  0  0
 0  0  1  0  1  0  0  0  0  0

In [20]:

Y = Vector(data.Y)
Z = Vector(data.Z)
XA = transpose(Matrix(dba[:, cols]))
XB = transpose(Matrix(dbb[:, cols]))
base = data.database

indA = findall(base .== 1)
indB = findall(base .== 2)

Xobserv = vcat(X[indA, :], X[indB, :])
Yobserv = vcat(Y[indA], Y[indB])
Zobserv = vcat(Z[indA], Z[indB])

nA = length(indA)
nB = length(indB)

# list the distinct modalities in A and B
indY = [findall(Y[indA] .== m) for m in Ylevels]
indZ = [findall(Z[indB] .== m) for m in Zlevels]

# compute the distance between pairs of individuals in different bases
# devectorize all the computations to go about twice faster only compute norm 1 here


6-element Vector{Vector{Int64}}:
 [26, 76, 678, 680, 694, 918, 951, 1022, 1549, 1790  …  2613, 2951, 3087, 3093, 3180, 3222, 3226, 3233, 3239, 3256]
 [1, 2, 5, 11, 19, 24, 30, 32, 33, 48  …  3317, 3318, 3334, 3338, 3341, 3342, 3344, 3345, 3348, 3352]
 [8, 10, 12, 14, 15, 16, 27, 31, 34, 36  …  3306, 3312, 3314, 3325, 3326, 3332, 3343, 3346, 3347, 3350]
 [717, 1198, 1199, 1200, 1201, 1202, 1203, 1206, 1207, 1208  …  1897, 1899, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1911]
 [3, 7, 9, 20, 21, 22, 23, 25, 35, 37  …  3328, 3331, 3333, 3335, 3336, 3337, 3340, 3349, 3351, 3354]
 [4, 6, 13, 17, 18, 28, 29, 40, 42, 45  …  3311, 3315, 3316, 3319, 3321, 3327, 3329, 3330, 3339, 3353]

In [21]:
struct GowerDF2 <: PreMetric
    continuous::Vector{Symbol}
    categorical::Vector{Symbol}
    ranges::Dict{Symbol,Float64}   # range par variable continue
end

function GowerDF2(continuous, categorical, df::DataFrame)
    ranges = Dict{Symbol,Float64}()
    for col in continuous
        coldata = skipmissing(df[!, col])
        ranges[col] = maximum(coldata) - minimum(coldata)
     end
    return GowerDF2(continuous, categorical, ranges)
end

function Distances.evaluate(d::GowerDF2, a::DataFrameRow, b::DataFrameRow)
    s = 0.0
    p = length(d.continuous) + length(d.categorical)

    # continues (version correcte Gower)
    for col in d.continuous
        range = d.ranges[col]

        if range == 0 || ismissing(a[col]) || ismissing(b[col])
            contrib = 0.0
        else
            contrib = abs(a[col] - b[col]) / range
        end

        s += contrib
    end

    # catégories
    for col in d.categorical
        if ismissing(a[col]) || ismissing(b[col])
            s += 0.0
        else
            s += (a[col] == b[col] ? 0.0 : 1.0)
        end
    end

    return s / p
end

function pairwise_gower(d::GowerDF2, A::DataFrame, B::DataFrame)
    nA = nrow(A)
    nB = nrow(B)
    D = Matrix{Float64}(undef, nA, nB)

    for i in 1:nA
        row_i = A[i, :]
        for j in 1:nB
            row_j = B[j, :]
            D[i,j] = Distances.evaluate(d, row_i, row_j)
        end
    end

    return D
end

pairwise_gower (generic function with 1 method)

In [23]:

cols = names(dba, r"^X")
for name in ["X2","X3"]
    lev = union(unique(dba[!, name]), unique(dbb[!, name]))
    dba[!, name] = categorical(dba[!, name], levels=lev)
    dbb[!, name] = categorical(dbb[!, name], levels=lev)
end             

# convertir les continues en Float64
for name in ["X1","X4"]
    dba[!, name] = Float64.(dba[!, name])
    dbb[!, name] = Float64.(dbb[!, name])
end
A = dba[:, cols]
B = dbb[:, cols]
C = vcat(A, B)
dist = GowerDF2([:X1,:X4], [:X2,:X3],C)  
# version pour DataFrameRow
D = pairwise_gower(dist, A, B)

991×3354 Matrix{Float64}:
 0.033361   0.0180593   0.0523346  …  0.296052   0.0659815  0.0613674
 0.0622442  0.0634874   0.0407995     0.284557   0.0234247  0.0335685
 0.0588925  0.0196169   0.084429      0.328146   0.0915131  0.0934617
 0.0596664  0.0203907   0.0563883     0.300106   0.0922869  0.0654211
 0.0540217  0.0538996   0.0224182     0.278661   0.0866423  0.0296491
 0.346865   0.30759     0.315262   …  0.0715043  0.379486   0.322493
 0.0316347  0.0389852   0.100785      0.344503   0.0824212  0.109818
 0.146078   0.147322    0.0769275     0.33321    0.0952919  0.0678948
 0.0330565  0.00686167  0.0772556     0.320973   0.065677   0.0862884
 0.130327   0.13157     0.10355       0.347307   0.0795403  0.0963188
 ⋮                                 ⋱                        
 0.0679295  0.0402995   0.0363259     0.292568   0.10055    0.0435568
 0.118653   0.119896    0.0495022     0.305785   0.0678665  0.0404694
 0.302087   0.339595    0.371237      0.114955   0.352873   0.38027
 0.2854

In [24]:

lambda = 0.1
alpha = 0.4
percent_closest = 1
#a = Xmdn[indA, :]'
#b = Xmdn[indB, :]'
#a = X_hot[indA, :]'
#b = X_hot[indB, :]'
distance = Euclidean()
#D = pairwise(distance, a, b, dims = 2)


    # Compute the indexes of individuals with same covariates
indXA = Vector{Int64}[]
indXB = Vector{Int64}[]

Xlevels = unique(eachrow(X))
# aggregate both bases
a = X[indA, :]'
b = X[indB, :]'
for x in Xlevels
        distA = vec(pairwise(distance, x[:, :], a, dims = 2))
        push!(indXA, findall(distA .< 0.1))
end
Xlevels = unique(eachrow(X))
for x in Xlevels
        distB = vec(pairwise(distance, x[:, :], b, dims = 2))
        push!(indXB, findall(distB .< 0.1))
end


norme = Euclidean()

aggregate_tol = 0.5

# Create a model for the optimal transport of individuals
modelA = Model(Clp.Optimizer)
modelB = Model(Clp.Optimizer)
set_optimizer_attribute(modelA, "LogLevel", 0)
set_optimizer_attribute(modelB, "LogLevel", 0)

# Compute data for aggregation of the individuals
Xlevels = eachindex(indXA)

# compute the neighbors of the covariates for regularization
Xvalues = unique(eachrow(Xobserv))
dist_X = pairwise(norme, Xvalues, Xvalues)
voisins = findall.(eachrow(dist_X .<= 1))
nvoisins = length(Xvalues)
C = zeros(Float64, (length(Ylevels), length(Zlevels)))

for y in Ylevels, i in indY[y], z in Zlevels

        nbclose = round(Int, percent_closest * length(indZ[z]))
        if nbclose > 0
            distance = [D[i, j] for j in indZ[z]]
            p = partialsortperm(distance, 1:nbclose)
            C[y, z] += sum(distance[p]) / nbclose / length(indY[y]) / 2.0
        end

end

for z in Zlevels, j in indZ[z], y in Ylevels

        nbclose = round(Int, percent_closest * length(indY[y]))
        if nbclose > 0
            distance = [D[i, j] for i in indY[y]]
            p = partialsortperm(distance, 1:nbclose)
            C[y, z] += sum(distance[p]) / nbclose / length(indZ[z]) / 2.0
        end

end

# Compute the estimators that appear in the model

estim_XA = length.(indXA) ./ nA
estim_XB = length.(indXB) ./ nB
estim_XA_YA = [
        length(indXA[x][findall(Yobserv[indXA[x]] .== y)]) / nA for
            x in Xlevels, y in Ylevels
    ]
estim_XB_ZB = [
        length(indXB[x][findall(Zobserv[indXB[x] .+ nA] .== z)]) / nB for
            x in Xlevels, z in Zlevels
    ]

    # Variables
    # - gammaA[x,y,z]: joint probability of X=x, Y=y and Z=z in base A
@variable(
        modelA,
        gammaA[x in Xlevels, y in Ylevels, z in Zlevels] >= 0,
        base_name = "gammaA"
    )

    # - gammaB[x,y,z]: joint probability of X=x, Y=y and Z=z in base B
@variable(
        modelB,
        gammaB[x in Xlevels, y in Ylevels, z in Zlevels] >= 0,
        base_name = "gammaB"
    )

@variable(modelA, errorA_XY[x in Xlevels, y in Ylevels], base_name = "errorA_XY")
@variable(
        modelA,
        abserrorA_XY[x in Xlevels, y in Ylevels] >= 0,
        base_name = "abserrorA_XY"
    )
@variable(modelA, errorA_XZ[x in Xlevels, z in Zlevels], base_name = "errorA_XZ")
@variable(
        modelA,
        abserrorA_XZ[x in Xlevels, z in Zlevels] >= 0,
        base_name = "abserrorA_XZ"
    )

@variable(modelB, errorB_XY[x in Xlevels, y in Ylevels], base_name = "errorB_XY")
@variable(
        modelB,
        abserrorB_XY[x in Xlevels, y in Ylevels] >= 0,
        base_name = "abserrorB_XY"
    )
@variable(modelB, errorB_XZ[x in Xlevels, z in Zlevels], base_name = "errorB_XZ")
@variable(
        modelB,
        abserrorB_XZ[x in Xlevels, z in Zlevels] >= 0,
        base_name = "abserrorB_XZ"
    )

# Constraints
# - assign sufficient probability to each class of covariates with the same outcome
@constraint(
        modelA,
        ctYandXinA[x in Xlevels, y in Ylevels],
        sum(gammaA[x, y, z] for z in Zlevels) == estim_XA_YA[x, y] + errorA_XY[x, y]
    )
@constraint(
        modelB,
        ctZandXinB[x in Xlevels, z in Zlevels],
        sum(gammaB[x, y, z] for y in Ylevels) == estim_XB_ZB[x, z] + errorB_XZ[x, z]
    )

    # - we impose that the probability of Y conditional to X is the same in the two databases
    # - the consequence is that the probability of Y and Z conditional to Y is also the same in the two bases
@constraint(
        modelA,
        ctZandXinA[x in Xlevels, z in Zlevels],
        estim_XB[x] * sum(gammaA[x, y, z] for y in Ylevels) ==
            estim_XB_ZB[x, z] * estim_XA[x] + estim_XB[x] * errorA_XZ[x, z]
    )

@constraint(
        modelB,
        ctYandXinB[x in Xlevels, y in Ylevels],
        estim_XA[x] * sum(gammaB[x, y, z] for z in Zlevels) ==
            estim_XA_YA[x, y] * estim_XB[x] + estim_XA[x] * errorB_XY[x, y]
    )

    # - recover the norm 1 of the error
@constraint(modelA, [x in Xlevels, y in Ylevels], errorA_XY[x, y] <= abserrorA_XY[x, y])
@constraint(
        modelA,
        [x in Xlevels, y in Ylevels],
        -errorA_XY[x, y] <= abserrorA_XY[x, y]
    )
@constraint(
        modelA,
        sum(abserrorA_XY[x, y] for x in Xlevels, y in Ylevels) <= alpha / 2.0
    )
@constraint(modelA, sum(errorA_XY[x, y] for x in Xlevels, y in Ylevels) == 0.0)
@constraint(modelA, [x in Xlevels, z in Zlevels], errorA_XZ[x, z] <= abserrorA_XZ[x, z])
@constraint(
        modelA,
        [x in Xlevels, z in Zlevels],
        -errorA_XZ[x, z] <= abserrorA_XZ[x, z]
    )

@constraint(
        modelA,
        sum(abserrorA_XZ[x, z] for x in Xlevels, z in Zlevels) <= alpha / 2.0
    )

@constraint(modelA, sum(errorA_XZ[x, z] for x in Xlevels, z in Zlevels) == 0.0)

@constraint(modelB, [x in Xlevels, y in Ylevels], errorB_XY[x, y] <= abserrorB_XY[x, y])

@constraint(
        modelB,
        [x in Xlevels, y in Ylevels],
        -errorB_XY[x, y] <= abserrorB_XY[x, y]
    )

@constraint(
        modelB,
        sum(abserrorB_XY[x, y] for x in Xlevels, y in Ylevels) <= alpha / 2.0
    )

@constraint(modelB, sum(errorB_XY[x, y] for x in Xlevels, y in Ylevels) == 0.0)

@constraint(modelB, [x in Xlevels, z in Zlevels], errorB_XZ[x, z] <= abserrorB_XZ[x, z])

@constraint(
        modelB,
        [x in Xlevels, z in Zlevels],
        -errorB_XZ[x, z] <= abserrorB_XZ[x, z]
    )

@constraint(
        modelB,
        sum(abserrorB_XZ[x, z] for x in Xlevels, z in Zlevels) <= alpha / 2.0
    )

@constraint(modelB, sum(errorB_XZ[x, z] for x in Xlevels, z in Zlevels) == 0.0)

    # - regularization
@variable(
        modelA,
        reg_absA[x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels] >= 0
    )

@constraint(
        modelA,
        [x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels],
        reg_absA[x1, x2, y, z] >=
            gammaA[x1, y, z] / (max(1, length(indXA[x1])) / nA) -
            gammaA[x2, y, z] / (max(1, length(indXA[x2])) / nA)
    )

    @constraint(
        modelA,
        [x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels],
        reg_absA[x1, x2, y, z] >=
            gammaA[x2, y, z] / (max(1, length(indXA[x2])) / nA) -
            gammaA[x1, y, z] / (max(1, length(indXA[x1])) / nA)
    )

@expression(
        modelA,
        regterm,
        sum(
            1 / nvoisins * reg_absA[x1, x2, y, z] for
                x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels
        )
    )

@variable(
        modelB,
        reg_absB[x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels] >= 0
    )

@constraint(
        modelB,
        [x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels],
        reg_absB[x1, x2, y, z] >=
            gammaB[x1, y, z] / (max(1, length(indXB[x1])) / nB) -
            gammaB[x2, y, z] / (max(1, length(indXB[x2])) / nB)
    )

@constraint(
        modelB,
        [x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels],
        reg_absB[x1, x2, y, z] >=
            gammaB[x2, y, z] / (max(1, length(indXB[x2])) / nB) -
            gammaB[x1, y, z] / (max(1, length(indXB[x1])) / nB)
    )

@expression(
        modelB,
        regterm,
        sum(
            1 / nvoisins * reg_absB[x1, x2, y, z] for
                x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels
        )
    )

    # by default, the OT cost and regularization term are weighted to lie in the same interval
@objective(
        modelA,
        Min,
        sum(C[y, z] * gammaA[x, y, z] for y in Ylevels, z in Zlevels, x in Xlevels) +
            lambda * sum(
            1 / nvoisins * reg_absA[x1, x2, y, z] for
                x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels
        )
    )

@objective(
        modelB,
        Min,
        sum(C[y, z] * gammaB[x, y, z] for y in Ylevels, z in Zlevels, x in Xlevels) +
            lambda * sum(
            1 / nvoisins * reg_absB[x1, x2, y, z] for
                x1 in Xlevels, x2 in voisins[x1], y in Ylevels, z in Zlevels
        )
    )

    # Solve the problem
optimize!(modelA)
optimize!(modelB)

    # Extract the values of the solution
gammaA_val = [value(gammaA[x, y, z]) for x in Xlevels, y in Ylevels, z in Zlevels]
gammaB_val = [value(gammaB[x, y, z]) for x in Xlevels, y in Ylevels, z in Zlevels]

    # compute the resulting estimators for the distributions of Z
    # conditional to X and Y in base A and of Y conditional to X and Z in base B

estimatorZA = ones(length(Xlevels), length(Ylevels), length(Zlevels)) ./ length(Zlevels)

for x in Xlevels, y in Ylevels
        proba_c_mA = sum(gammaA_val[x, y, Zlevels])
        if proba_c_mA > 1.0e-6
            estimatorZA[x, y, :] = gammaA_val[x, y, :] ./ proba_c_mA
        end
end

estimatorYB = ones(length(Xlevels), length(Ylevels), length(Zlevels)) ./ length(Ylevels)

for x in Xlevels, z in Zlevels
        proba_c_mB = sum(view(gammaB_val, x, Ylevels, z))
        if proba_c_mB > 1.0e-6
            estimatorYB[x, :, z] = view(gammaB_val, x, :, z) ./ proba_c_mB
        end
end

nbX = length(indXA)

    # Count the number of mistakes in the transport
    #deduce the individual distributions of probability for each individual from the distributions
probaZindivA = zeros(Float64, (nA, length(Zlevels)))
probaYindivB = zeros(Float64, (nB, length(Ylevels)))
for x in 1:nbX
        for i in indXA[x]
            probaZindivA[i, :] .= estimatorZA[x, Yobserv[i], :]
end
for i in indXB[x]
            probaYindivB[i, :] .= estimatorYB[x, :, Zobserv[i + nA]]
        end
end

    # Transport the modality that maximizes frequency
predZA = [findmax([probaZindivA[i, z] for z in Zlevels])[2] for i in 1:nA]
predYB = [findmax([probaYindivB[j, y] for y in Ylevels])[2] for j in 1:nB]



3354-element Vector{Int64}:
 3
 3
 3
 5
 3
 5
 3
 3
 3
 3
 ⋮
 3
 5
 3
 3
 4
 3
 3
 5
 3

In [32]:

tab1 = FreqTables.freqtable(dba.Y,predZA)
tab2 = FreqTables.freqtable(dbb.Z,predYB)
@show tab2

#Y codage timoun 1 2 3 4 5
#Y1 aucune etude ==> Z1
#Y2 primaire ==> Z1
#Y3 secondaire ==> Z2
#Y4 baccalaureat ==> Z3
#Y5 supérieures ==> Z4 Z5 Z6

#Z codage pelagie 1 2 3 4 5 6
#Z1 Aucune scolarisation,  études primaires ==> Y1 Y2
#Z2 Une partie des études secondaires ==> Y3
#Z3 Niveau baccalauréat ==> Y4
#Z4 études supérieures ==> Y5
#Z5 Bac + 2 ==> Y5
#Z6 Bac + 3 et plus ==> Y5
accZ=(tab1[1,1]+tab1[2,1]+tab1[3,2]+tab1[4,3]+tab1[5,4]+tab1[5,5])/sum(tab1)

#accY=(tab2[1,1]+tab2[1,2]+tab2[2,3]+tab2[3,4]+tab2[4,5]+tab2[5,5]+tab2[6,5])/sum(tab2)

@show accZ

tab2 = [19 0 0 5 0; 0 0 553 11 28; 0 0 572 21 22; 0 2 91 0 0; 0 0 834 0 42; 0 0 4 485 665]
accZ = 0.4066599394550959


0.4066599394550959

In [34]:
T = Float64
X_hot=Matrix{T}(X_hot)
Xdf = DataFrame(X_hot, Symbol.("X" .* string.(1:size(X_hot, 2))))
data2 = hcat(Xdf,data[:, [:Y, :Z, :database]])
dba = subset(data2, :database => ByRow(==(1)))
dbb = subset(data2, :database => ByRow(==(2)))
@show data2




Row,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,Y,Z,database
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,4,-1,1
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4,-1,1
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,5,-1,1
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3,-1,1
5,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,4,-1,1
6,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,4,-1,1
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3,-1,1
8,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3,-1,1
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3,-1,1


In [36]:
A= otrecod(data2, JointOTBetweenBases(reg = 0.001,
reg_m1= 0.01,
reg_m2 = 0.01,
Ylevels = 1:5,
Zlevels = 1:6))

@show A.yb_pred
@show A.za_pred

991-element Vector{Int64}:
 6
 6
 2
 6
 6
 6
 2
 5
 2
 5
 ⋮
 6
 5
 2
 6
 5
 6
 6
 6
 2

In [39]:

tab1 = FreqTables.freqtable(dba.Y,predZA)
tab2 = FreqTables.freqtable(dbb.Z,predYB)
@show tab2

#Y codage timoun 1 2 3 4 5
#Y1 aucune etude ==> Z1
#Y2 primaire ==> Z1
#Y3 secondaire ==> Z2
#Y4 baccalaureat ==> Z3
#Y5 supérieures ==> Z4 Z5 Z6

#Z codage pelagie 1 2 3 4 5 6
#Z1 Aucune scolarisation,  études primaires ==> Y1 Y2
#Z2 Une partie des études secondaires ==> Y3
#Z3 Niveau baccalauréat ==> Y4
#Z4 études supérieures ==> Y5
#Z5 Bac + 2 ==> Y5
#Z6 Bac + 3 et plus ==> Y5
accZ=(tab1[1,1]+tab1[2,1]+tab1[3,2]+tab1[4,3]+tab1[5,4]+tab1[5,5])/sum(tab1)

accY=(tab2[1,1]+tab2[1,2]+tab2[2,3]+tab2[3,4]+tab2[4,5]+tab2[5,5]+tab2[6,5])/sum(tab2)

@show accZ

0.4066599394550959